# Load Libraries

In [ ]:
from IPython.display import display, HTML
from pystripe.core import *
import matplotlib.pyplot as plt

from supplements.croping import imwrite_lzw

display(HTML("<style>.container { width:100% !important; }</style>"))

def plot_images(img_list: List[ndarray], img_labels: List[str], vmax: int):
    fig, axes = plt.subplots(nrows=1, ncols=len(img_list), figsize=(20, 20))
    for idx, (im, label) in enumerate(zip(img_list, img_labels)):
        axes[idx].imshow(im, cmap='gray', vmin=0, vmax=vmax)
        axes[idx].set_title(label)
    plt.tight_layout()
    plt.show()
img = imread_tif_raw_png(Path(r"C:\Users\kmoradi\Downloads\test.png"))

# Test Deconvolution

In [ ]:
# Testing deconvolutions for temp test folder (containing 1 image)

from nrrd import read, write
from tifffile import imwrite
import shutil, os
import time
from pathlib import Path
from PIL import Image

# # delete old directory
# try:
#     shutil.rmtree(r"E:\TMP\Aidan\out_test_single\temp")
#     os.remove(r"E:\TMP\Aidan\out_test_single\y00006400.x00019200.nrrd")
# except FileNotFoundError:
#     print("File not found")
# 
# print("Old directory deleted")
# time.sleep(0.5)
# 
# # run fnt_cube_processor.py
! python .\fnt_cube_processor.py -i E:\TMP\Keivan\decon_test_input -o E:\TMP\Keivan\decon_test_output -n 1 -d -dxy .7 -dz 1.2 -ds -g 1 -it 12 -dgi 3 -cef 1 -dpsf
# # time.sleep(2)
output = Path(r"E:\TMP\Keivan\decon_test_output")
for file in output.glob("*.nrrd"):
    img, header = read(file)
    # imwrite(output / (file.stem + ".tif"), img, compression="LZW")
    image_stack = [Image.fromarray(_) for _ in img]
    image_stack[0].save(output / (file.stem + ".tif"), save_all=True, append_images=image_stack[1:], compression='tiff_lzw')
    file.unlink()

# Compare two groups of neuron names to see which one is missing in each group

In [ ]:
import pandas as pd
from pathlib import Path
SWCs = Path(r"Y:\3D_stitched_LS\20230624_SM230223_05_LS_15x_800z_Double_MORF_B4\Ex_642_Em_690_Ano\QC2\SWCs")
tracker = Path(r"D:\kmoradi\Documents\Downloads\KM_SN) SM230323-05 (B4) Tracker - Tracker.csv")
df = pd.read_csv(tracker, skiprows=1)
df = df[~df.Unreconstructable]
# df = df[df.QC1]
df = df[~pd.isna(df.Reconstructor)]
#df = df[df.Channel == "Ex_488_Em_525"]
df = df[df.Channel == "Ex_642_Em_690"]
neuron_ids = set(df.Neuron_ID)
files = set(tuple([file.stem for file in SWCs.glob("*.swc")] + [file.stem for file in SWCs.glob("*.fnt")]))
print("neuron in tracker missing on QC2")
for file in neuron_ids.difference(files):
    print(file)
print("neuron in QC2 missing on tracker")
for file in files.difference(neuron_ids):
    print(file)

# Find seeds that are not reconstructed already

In [ ]:
from pathlib import Path
SWCs = Path(r"X:\3D_stitched_LS\20231010_FM230407_07_LS_15x_800z_AD1\Ex_642_Ch2_Ano\QC2\SWCs")
Seeds = Path(r"X:\3D_stitched_LS\20231010_FM230407_07_LS_15x_800z_AD1\Ex_642_Ch2_Ano\Seeds\Unsorted")
files = set(tuple([file.stem for file in SWCs.glob("*.swc")]))
seeds = set(tuple([file.stem for file in Seeds.glob("*.swc")]))
print(f"{len(seeds.intersection(files))} neuron in Seeds folder that are already reconstructed")
for file in seeds.intersection(files):
    (Seeds / (file + ".swc")).unlink()
    # print(file)
# print(f"{len(files.difference(seeds))} neuron in QC2 missing in Seeds folder")
# for file in files.difference(seeds):
#     (Seeds / (file + ".swc")).unlink()
    # print(file)